In [14]:
import models
from core.config import config
from core.config import update_config
from core.config import update_dir
import torch
from torchvision import transforms
import os
import random
from PIL import Image

In [2]:
cfg_pth = "../experiments/anime/resnet50/256x256_d256x3_adam_lr1e-3.yaml"
update_config(cfg_pth)

In [3]:
model = torch.nn.DataParallel(models.pose_resnet.get_pose_net(config, is_train=False))

In [4]:
model.eval()

DataParallel(
  (module): PoseResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
  

In [5]:
ckpt_file = "../output/anime/pose_resnet_50/256x256_d256x3_adam_lr1e-3/checkpoint.pth.tar"
checkpoint = torch.load(ckpt_file)

model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [21]:
base_dir ="../data/animepose/images/"
images = os.listdir(base_dir)
filename = base_dir + random.choice(images)

input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(360),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.7137, 0.6628, 0.6519], std=[0.2970, 0.3017, 0.2979]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)

device = torch.device("cuda:0")
input_batch = input_batch.to(device)
model = model.to(device)

In [22]:
with torch.no_grad():
    output = model(input_batch)

In [23]:
print(output.size())

torch.Size([1, 21, 128, 96])
